In [2]:
import numpy as np
import pickle
import datetime
import pandas as pd

In [3]:
#变量映射
variable_mapping = dict()
variable_mapping['FiO2'] = [226754,227009,227010]
variable_mapping['resp'] = [220210, 224688, 224689, 224690]
variable_mapping['po2'] = [226770,227039,220224,227516]
variable_mapping['pco2'] = [220235,227036,226063,226062]
variable_mapping['temp'] = [223761, 223762, 224027]
variable_mapping['hr'] = [220045]
# variable_mapping['BP'] = [220050, 220059, 220179, 224167, 225309, 227243, 226850, 226852]
variable_mapping['ABP'] = [220050] # 动脉收缩压
variable_mapping['K'] = [220640, 227464, 227442, 226772, 226535]
variable_mapping['Na'] = [220645, 226534, 226776]
variable_mapping['hco3'] = [224826, 226759, 227443]
variable_mapping['spo2'] = [220227, 220277, 226860,226861,226862,226863,226865,228232]
variable_mapping['BUN'] = [225624,227000,227001]
variable_mapping['airway'] = [223838, 224832, 224391, 227810,223837, 224829]
variable_mapping['gcs'] = [220739, 223900, 223901, 226755, 226756, 226757, 226758, 227011, 227012, 227013, 227014,228112]
variable_mapping['creatinine'] = [220615, 226752, 227005,226751]
variable_mapping['pain'] = [223791, 227881]
variable_mapping['Urine'] = [227519, 227059]
variable_mapping['Hemoglobin'] = [220228]
variable_mapping['peep'] = [220339, 224699, 224700]


variable_mapping['MEANBP'] = [220052]
variable_mapping['RR'] = [226773,226774,227050,227051,220210]
variable_mapping['SaO2'] = [224719]
variable_mapping['HCT']=[220545,226540,226761,226762,227017]
variable_mapping['RBC'] = [226368,227070]
variable_mapping['ALB'] = [227456,226981,226982]
variable_mapping['ALT'] = [220644]
variable_mapping['AST'] = [220587]            
variable_mapping['WBC'] = [220546,226779,226780,227062,227063]  
variable_mapping['PLT'] = [225170,227457,226369,227071]
variable_mapping['NEUT'] = [225643]
variable_mapping['TBIL'] = [225690]                    
variable_mapping['PTT'] = [227466]
variable_mapping['PT'] = [227465]
variable_mapping['INR'] = [227467]                    
variable_mapping['GLU'] = [220621,227015]
variable_mapping['THR'] = [227469]
variable_mapping['DIASBP'] = [226851,226853]
variable_mapping['CO2'] = [220245]
variable_mapping['CL'] = [220602]
variable_mapping['Lac'] = [220955]
variable_mapping['MV'] = [226260]

In [4]:
#itemid 编号、模块、主键。。。
def _remove_anomalies(data, physical_limit,index, variable_dict, verbose=False):
    ## define which ITEMIDs correspond to which variables.定义哪些itemid对应于哪些变量。
    if verbose:
        print ("For example:")
        # removed = data[data['remove_col']==1].sample(0.001)
        removed = data[data['remove_col']==1].sample(frac=0.001, replace=True, random_state=1)
        # removed.print_rows(num_rows=20)
        num_rows = removed.iloc[0:20]
        print(num_rows)
    data = data.loc[index]
    data=data.drop(['remove_col'], axis=1)
    print ("Anomalies removed.")
    return data

In [5]:
all_data = pd.read_csv('all_data_DeathOrNot.csv',index_col=0)
all_data

E:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (6,8,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,C.SUBJECT_ID,C.HADM_ID,C.ICUSTAY_ID,D.ITEMID,C.CHARTTIME,C.VALUE,C.VALUENUM,C.VALUEUOM,II.INTIME,II.OUTTIME,II.LOS,D.LABEL,D.UNITNAME,hrs_bd,final_24hr,final_48hr,final_72hr,Death
0,109,147469,253139,220045,2141-06-11 12:13:00,93.0,93.0,bpm,2141-06-11 10:17:54,2141-06-15 15:11:37,4.2040,Heart Rate,bpm,98.976944,1,1,1,0
1,109,147469,253139,220045,2141-06-11 12:15:00,97.0,97.0,bpm,2141-06-11 10:17:54,2141-06-15 15:11:37,4.2040,Heart Rate,bpm,98.943611,1,1,1,0
2,109,147469,253139,220045,2141-06-11 12:30:00,105.0,105.0,bpm,2141-06-11 10:17:54,2141-06-15 15:11:37,4.2040,Heart Rate,bpm,98.693611,1,1,1,0
3,109,147469,253139,220045,2141-06-11 12:46:00,84.0,84.0,bpm,2141-06-11 10:17:54,2141-06-15 15:11:37,4.2040,Heart Rate,bpm,98.426944,1,1,1,0
4,109,147469,253139,220045,2141-06-11 13:00:00,84.0,84.0,bpm,2141-06-11 10:17:54,2141-06-15 15:11:37,4.2040,Heart Rate,bpm,98.193611,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3967270,17190,106415,211988,227881,2107-12-20 10:26:00,10 / 10,NaN,NaN,2107-11-15 11:04:22,2107-12-21 16:40:53,36.2337,Pain (0-10),NaN,30.248056,0,0,0,0
3967271,40978,151069,222678,227881,2177-08-22 11:30:00,0 / 10,NaN,NaN,2177-08-11 18:45:34,2177-08-28 18:55:08,17.0066,Pain (0-10),NaN,151.418889,0,0,0,0
3967272,67675,125918,202918,227881,2183-10-08 10:43:00,0 / 10,NaN,NaN,2183-10-07 14:54:22,2183-10-08 17:59:52,1.1288,Pain (0-10),NaN,7.281111,1,1,1,0
3967273,73074,190770,252433,227881,2158-09-04 10:39:00,0 / 10,NaN,NaN,2158-09-02 20:10:38,2158-09-04 14:56:44,1.7820,Pain (0-10),NaN,4.295556,0,1,1,1


In [8]:
# all_data[all_data['D.LABEL']=='temp']

In [7]:
all_data = all_data.dropna(axis=0, how="any", subset=['D.ITEMID','C.VALUENUM'], inplace=False)
all_data

,C.SUBJECT_ID,C.HADM_ID,C.ICUSTAY_ID,D.ITEMID,C.CHARTTIME,C.VALUE,C.VALUENUM,C.VALUEUOM,II.INTIME,II.OUTTIME,II.LOS,D.LABEL,D.UNITNAME,hrs_bd,final_24hr,final_48hr,final_72hr,Death
0,109,147469,253139,220045,2141-06-11 12:13:00,93.0,93.0,bpm,2141-06-11 10:17:54,2141-06-15 15:11:37,4.2040,Heart Rate,bpm,98.976944,1,1,1,0
1,109,147469,253139,220045,2141-06-11 12:15:00,97.0,97.0,bpm,2141-06-11 10:17:54,2141-06-15 15:11:37,4.2040,Heart Rate,bpm,98.943611,1,1,1,0
2,109,147469,253139,220045,2141-06-11 12:30:00,105.0,105.0,bpm,2141-06-11 10:17:54,2141-06-15 15:11:37,4.2040,Heart Rate,bpm,98.693611,1,1,1,0
3,109,147469,253139,220045,2141-06-11 12:46:00,84.0,84.0,bpm,2141-06-11 10:17:54,2141-06-15 15:11:37,4.2040,Heart Rate,bpm,98.426944,1,1,1,0
4,109,147469,253139,220045,2141-06-11 13:00:00,84.0,84.0,bpm,2141-06-11 10:17:54,2141-06-15 15:11:37,4.2040,Heart Rate,bpm,98.193611,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3967264,98254,114329,237545,228232,2107-10-18 12:06:00,Able to maintain O2 sat >92% on RA,2.0,NaN,2107-10-17 00:29:52,2107-10-18 18:35:58,1.7542,PAR-Oxygen saturation,NaN,6.499444,0,1,1,0
3967265,98724,186754,205000,228232,2143-08-23 16:56:00,Needs O2 inhalation to maintain O2 sat > 90%,1.0,NaN,2143-08-22 16:02:26,2143-08-24 19:58:56,2.1642,PAR-Oxygen saturation,NaN,27.048889,0,1,1,0
3967266,99923,192053,298857,228232,2201-05-16 06:00:00,Needs O2 inhalation to maintain O2 sat > 90%,1.0,NaN,2201-05-16 06:06:34,2201-05-18 11:45:27,2.2353,PAR-Oxygen saturation,NaN,53.757500,1,1,1,0
3967267,99923,192053,298857,228232,2201-05-16 06:15:00,Needs O2 inhalation to maintain O2 sat > 90%,1.0,NaN,2201-05-16 06:06:34,2201-05-18 11:45:27,2.2353,PAR-Oxygen saturation,NaN,53.507500,1,1,1,0


In [8]:
# 有多少个细分的生物标志物
len(list(all_data['D.ITEMID'].unique()))

63

In [24]:
get_var = lambda it: [variable for variable,items in variable_mapping.items() if it in items][0]
all_data['VARIABLES'] = all_data['D.ITEMID'].apply(lambda itemid: get_var(itemid))
all_data

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,C.SUBJECT_ID,C.HADM_ID,C.ICUSTAY_ID,D.ITEMID,C.CHARTTIME,C.VALUE,C.VALUENUM,C.VALUEUOM,II.INTIME,II.OUTTIME,II.LOS,D.LABEL,D.UNITNAME,hrs_bd,final_24hr,final_48hr,final_72hr,Death,VARIABLES
0,109,147469,253139,220045,2141-06-11 12:13:00,93.0,93.0,bpm,2141-06-11 10:17:54,2141-06-15 15:11:37,4.2040,Heart Rate,bpm,98.976944,1,1,1,0,hr
1,109,147469,253139,220045,2141-06-11 12:15:00,97.0,97.0,bpm,2141-06-11 10:17:54,2141-06-15 15:11:37,4.2040,Heart Rate,bpm,98.943611,1,1,1,0,hr
2,109,147469,253139,220045,2141-06-11 12:30:00,105.0,105.0,bpm,2141-06-11 10:17:54,2141-06-15 15:11:37,4.2040,Heart Rate,bpm,98.693611,1,1,1,0,hr
3,109,147469,253139,220045,2141-06-11 12:46:00,84.0,84.0,bpm,2141-06-11 10:17:54,2141-06-15 15:11:37,4.2040,Heart Rate,bpm,98.426944,1,1,1,0,hr
4,109,147469,253139,220045,2141-06-11 13:00:00,84.0,84.0,bpm,2141-06-11 10:17:54,2141-06-15 15:11:37,4.2040,Heart Rate,bpm,98.193611,1,1,1,0,hr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3967264,98254,114329,237545,228232,2107-10-18 12:06:00,Able to maintain O2 sat >92% on RA,2.0,NaN,2107-10-17 00:29:52,2107-10-18 18:35:58,1.7542,PAR-Oxygen saturation,NaN,6.499444,0,1,1,0,spo2
3967265,98724,186754,205000,228232,2143-08-23 16:56:00,Needs O2 inhalation to maintain O2 sat > 90%,1.0,NaN,2143-08-22 16:02:26,2143-08-24 19:58:56,2.1642,PAR-Oxygen saturation,NaN,27.048889,0,1,1,0,spo2
3967266,99923,192053,298857,228232,2201-05-16 06:00:00,Needs O2 inhalation to maintain O2 sat > 90%,1.0,NaN,2201-05-16 06:06:34,2201-05-18 11:45:27,2.2353,PAR-Oxygen saturation,NaN,53.757500,1,1,1,0,spo2
3967267,99923,192053,298857,228232,2201-05-16 06:15:00,Needs O2 inhalation to maintain O2 sat > 90%,1.0,NaN,2201-05-16 06:06:34,2201-05-18 11:45:27,2.2353,PAR-Oxygen saturation,NaN,53.507500,1,1,1,0,spo2


In [27]:
all_data = all_data.rename(columns={'VARIABLES':'VARIABLE'})
all_data

,C.SUBJECT_ID,C.HADM_ID,C.ICUSTAY_ID,D.ITEMID,C.CHARTTIME,C.VALUE,C.VALUENUM,C.VALUEUOM,II.INTIME,II.OUTTIME,II.LOS,D.LABEL,D.UNITNAME,hrs_bd,final_24hr,final_48hr,final_72hr,Death,VARIABLE
0,109,147469,253139,220045,2141-06-11 12:13:00,93.0,93.0,bpm,2141-06-11 10:17:54,2141-06-15 15:11:37,4.2040,Heart Rate,bpm,98.976944,1,1,1,0,hr
1,109,147469,253139,220045,2141-06-11 12:15:00,97.0,97.0,bpm,2141-06-11 10:17:54,2141-06-15 15:11:37,4.2040,Heart Rate,bpm,98.943611,1,1,1,0,hr
2,109,147469,253139,220045,2141-06-11 12:30:00,105.0,105.0,bpm,2141-06-11 10:17:54,2141-06-15 15:11:37,4.2040,Heart Rate,bpm,98.693611,1,1,1,0,hr
3,109,147469,253139,220045,2141-06-11 12:46:00,84.0,84.0,bpm,2141-06-11 10:17:54,2141-06-15 15:11:37,4.2040,Heart Rate,bpm,98.426944,1,1,1,0,hr
4,109,147469,253139,220045,2141-06-11 13:00:00,84.0,84.0,bpm,2141-06-11 10:17:54,2141-06-15 15:11:37,4.2040,Heart Rate,bpm,98.193611,1,1,1,0,hr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3967264,98254,114329,237545,228232,2107-10-18 12:06:00,Able to maintain O2 sat >92% on RA,2.0,NaN,2107-10-17 00:29:52,2107-10-18 18:35:58,1.7542,PAR-Oxygen saturation,NaN,6.499444,0,1,1,0,spo2
3967265,98724,186754,205000,228232,2143-08-23 16:56:00,Needs O2 inhalation to maintain O2 sat > 90%,1.0,NaN,2143-08-22 16:02:26,2143-08-24 19:58:56,2.1642,PAR-Oxygen saturation,NaN,27.048889,0,1,1,0,spo2
3967266,99923,192053,298857,228232,2201-05-16 06:00:00,Needs O2 inhalation to maintain O2 sat > 90%,1.0,NaN,2201-05-16 06:06:34,2201-05-18 11:45:27,2.2353,PAR-Oxygen saturation,NaN,53.757500,1,1,1,0,spo2
3967267,99923,192053,298857,228232,2201-05-16 06:15:00,Needs O2 inhalation to maintain O2 sat > 90%,1.0,NaN,2201-05-16 06:06:34,2201-05-18 11:45:27,2.2353,PAR-Oxygen saturation,NaN,53.507500,1,1,1,0,spo2


In [29]:
all_data.to_csv('all_data_clean1.csv')

In [69]:
all_data = pd.read_csv('all_data_clean1.csv',index_col=0)
all_data
## 查看一下
print(len(all_data[all_data['C.SUBJECT_ID']==109]))  # 6206  条数据
print(len(all_data[all_data['C.SUBJECT_ID']==109]['D.ITEMID'].drop_duplicates()))  # 35 个检测变量

E:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (6,8,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


6206
35


In [87]:
all_data['C.ICUSTAY_ID'].drop_duplicates()

0          253139
108        243978
134        257358
171        262652
219        236124
            ...  
1974068    295845
1975039    285019
1998051    278458
2039288    210313
2269981    282730
Name: C.ICUSTAY_ID, Length: 5186, dtype: int64

# 取出【每个 C.SUBJECT_ID 中入院时间最小的 C.ICUSTAY_ID ，即 每个病人的第一次入院】

In [63]:
# all_data.groupby(['C.SUBJECT_ID'],as_index=False).agg({'II.INTIME': [np.min]})

,C.SUBJECT_ID,II.INTIME
,,amin
0,109,2141-06-11 10:17:54
1,111,2144-07-01 04:13:59
2,124,2166-01-09 13:05:18
3,188,2160-11-25 21:57:05
4,222,2143-02-16 12:59:06
...,...,...
3266,99899,2113-06-04 15:56:39
3267,99923,2201-02-23 20:43:12
3268,99939,2110-03-22 04:05:55


In [77]:
all_data[['C.SUBJECT_ID','II.INTIME','C.ICUSTAY_ID']].groupby(by='C.SUBJECT_ID',as_index=False).min('II.INTIME')

,C.SUBJECT_ID,C.ICUSTAY_ID
0,109,222630
1,111,249202
2,124,256064
3,188,213646
4,222,256144
...,...,...
3266,99899,245756
3267,99923,202505
3268,99939,252442
3269,99973,275083


In [75]:
First_ICU = all_data[['C.SUBJECT_ID','II.INTIME']].groupby(by='C.SUBJECT_ID',as_index=False).min()
First_ICU

,C.SUBJECT_ID,II.INTIME
0,109,2141-06-11 10:17:54
1,111,2144-07-01 04:13:59
2,124,2166-01-09 13:05:18
3,188,2160-11-25 21:57:05
4,222,2143-02-16 12:59:06
...,...,...
3266,99899,2113-06-04 15:56:39
3267,99923,2201-02-23 20:43:12
3268,99939,2110-03-22 04:05:55
3269,99973,2180-11-27 02:30:43


In [84]:
First_ICU = First_ICU.merge(all_data,how='left',on=['C.SUBJECT_ID','II.INTIME'])
First_ICU

,C.SUBJECT_ID,II.INTIME,C.HADM_ID,C.ICUSTAY_ID,D.ITEMID,C.CHARTTIME,C.VALUE,C.VALUENUM,C.VALUEUOM,II.OUTTIME,II.LOS,D.LABEL,D.UNITNAME,hrs_bd,final_24hr,final_48hr,final_72hr,Death,VARIABLE
0,109,2141-06-11 10:17:54,147469,253139,220045,2141-06-11 12:13:00,93.0,93.0,bpm,2141-06-15 15:11:37,4.2040,Heart Rate,bpm,98.976944,1,1,1,0,hr
1,109,2141-06-11 10:17:54,147469,253139,220045,2141-06-11 12:15:00,97.0,97.0,bpm,2141-06-15 15:11:37,4.2040,Heart Rate,bpm,98.943611,1,1,1,0,hr
2,109,2141-06-11 10:17:54,147469,253139,220045,2141-06-11 12:30:00,105.0,105.0,bpm,2141-06-15 15:11:37,4.2040,Heart Rate,bpm,98.693611,1,1,1,0,hr
3,109,2141-06-11 10:17:54,147469,253139,220045,2141-06-11 12:46:00,84.0,84.0,bpm,2141-06-15 15:11:37,4.2040,Heart Rate,bpm,98.426944,1,1,1,0,hr
4,109,2141-06-11 10:17:54,147469,253139,220045,2141-06-11 13:00:00,84.0,84.0,bpm,2141-06-15 15:11:37,4.2040,Heart Rate,bpm,98.193611,1,1,1,0,hr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2207256,99992,2144-07-25 18:04:42,197084,242052,223791,2144-07-27 00:00:00,Mild,2.0,NaN,2144-07-27 17:27:55,1.9745,Pain Level,NaN,17.465278,0,1,1,0,pain
2207257,99992,2144-07-25 18:04:42,197084,242052,223791,2144-07-27 04:00:00,Mild,2.0,NaN,2144-07-27 17:27:55,1.9745,Pain Level,NaN,13.465278,0,1,1,0,pain
2207258,99992,2144-07-25 18:04:42,197084,242052,223791,2144-07-27 08:00:00,Mild,2.0,NaN,2144-07-27 17:27:55,1.9745,Pain Level,NaN,9.465278,0,1,1,0,pain
2207259,99992,2144-07-25 18:04:42,197084,242052,223791,2144-07-27 12:00:00,None,0.0,NaN,2144-07-27 17:27:55,1.9745,Pain Level,NaN,5.465278,0,1,1,0,pain


In [88]:
First_ICU.to_csv('First_ICU.csv')

In [89]:
First_ICU = pd.read_csv('First_ICU.csv',index_col=0)
First_ICU

E:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,C.SUBJECT_ID,II.INTIME,C.HADM_ID,C.ICUSTAY_ID,D.ITEMID,C.CHARTTIME,C.VALUE,C.VALUENUM,C.VALUEUOM,II.OUTTIME,II.LOS,D.LABEL,D.UNITNAME,hrs_bd,final_24hr,final_48hr,final_72hr,Death,VARIABLE
0,109,2141-06-11 10:17:54,147469,253139,220045,2141-06-11 12:13:00,93.0,93.0,bpm,2141-06-15 15:11:37,4.2040,Heart Rate,bpm,98.976944,1,1,1,0,hr
1,109,2141-06-11 10:17:54,147469,253139,220045,2141-06-11 12:15:00,97.0,97.0,bpm,2141-06-15 15:11:37,4.2040,Heart Rate,bpm,98.943611,1,1,1,0,hr
2,109,2141-06-11 10:17:54,147469,253139,220045,2141-06-11 12:30:00,105.0,105.0,bpm,2141-06-15 15:11:37,4.2040,Heart Rate,bpm,98.693611,1,1,1,0,hr
3,109,2141-06-11 10:17:54,147469,253139,220045,2141-06-11 12:46:00,84.0,84.0,bpm,2141-06-15 15:11:37,4.2040,Heart Rate,bpm,98.426944,1,1,1,0,hr
4,109,2141-06-11 10:17:54,147469,253139,220045,2141-06-11 13:00:00,84.0,84.0,bpm,2141-06-15 15:11:37,4.2040,Heart Rate,bpm,98.193611,1,1,1,0,hr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2207256,99992,2144-07-25 18:04:42,197084,242052,223791,2144-07-27 00:00:00,Mild,2.0,NaN,2144-07-27 17:27:55,1.9745,Pain Level,NaN,17.465278,0,1,1,0,pain
2207257,99992,2144-07-25 18:04:42,197084,242052,223791,2144-07-27 04:00:00,Mild,2.0,NaN,2144-07-27 17:27:55,1.9745,Pain Level,NaN,13.465278,0,1,1,0,pain
2207258,99992,2144-07-25 18:04:42,197084,242052,223791,2144-07-27 08:00:00,Mild,2.0,NaN,2144-07-27 17:27:55,1.9745,Pain Level,NaN,9.465278,0,1,1,0,pain
2207259,99992,2144-07-25 18:04:42,197084,242052,223791,2144-07-27 12:00:00,None,0.0,NaN,2144-07-27 17:27:55,1.9745,Pain Level,NaN,5.465278,0,1,1,0,pain


# 第一次入院的患者提取完毕(上面)

In [90]:
all_data_subID = First_ICU['C.SUBJECT_ID'].drop_duplicates() # 3271 个患者
all_data_subID

0            109
579          111
689          124
4574         188
4684         222
           ...  
2204802    99899
2206060    99923
2206153    99939
2206725    99973
2207044    99992
Name: C.SUBJECT_ID, Length: 3271, dtype: int64

In [116]:
Age_Gender=pd.read_csv('C:/Users/ASUS/急性肾损伤数据提取/data/Age_Gender_subId.csv')

Age_Gender

,subject_id,hadm_id,age,SUBJECT_ID,GENDER
0,3,145834,76,3,M
1,9,150750,41,9,M
2,21,109451,87,21,M
3,38,185910,76,38,M
4,52,190797,39,52,M
...,...,...,...,...,...
9168,99982,112748,66,99982,M
9169,99982,183791,66,99982,M
9170,99983,117390,79,99983,M
9171,99991,151118,47,99991,M


In [117]:
Age_Gender = Age_Gender.drop(columns='SUBJECT_ID')
Age_Gender = Age_Gender.rename(columns={'subject_id':'SUBJECT_ID'})
Age_Gender

,SUBJECT_ID,hadm_id,age,GENDER
0,3,145834,76,M
1,9,150750,41,M
2,21,109451,87,M
3,38,185910,76,M
4,52,190797,39,M
...,...,...,...,...
9168,99982,112748,66,M
9169,99982,183791,66,M
9170,99983,117390,79,M
9171,99991,151118,47,M


In [118]:
Age_Gender = Age_Gender.drop(columns='hadm_id')

In [120]:
Min_Age = Age_Gender[['SUBJECT_ID','age']].groupby(by='SUBJECT_ID',as_index=False).min()
Age_Gender = Min_Age.merge(Age_Gender,on=['SUBJECT_ID','age'],how='left')

In [121]:
Age_Gender['SUBJECT_ID'].drop_duplicates()   # 7735 个患者

0           3
1           9
2          21
3          38
4          52
        ...  
8268    99973
8269    99982
8271    99983
8272    99991
8273    99992
Name: SUBJECT_ID, Length: 7735, dtype: int64

In [122]:
Age_Gender = Age_Gender.drop_duplicates()
Age_Gender

,SUBJECT_ID,age,GENDER
0,3,76,M
1,9,41,M
2,21,87,M
3,38,76,M
4,52,39,M
...,...,...,...
8268,99973,65,F
8269,99982,66,M
8271,99983,79,M
8272,99991,47,M


## 感觉这里有点问题，将性别和年龄拼接起来数据反而变多了

In [123]:
# all_data1 = pd.merge(First_ICU,Age_Gender,left_on=['C.SUBJECT_ID','C.HADM_ID'],right_on=['SUBJECT_ID','hadm_id'],how='left')
all_data1 = pd.merge(First_ICU,Age_Gender,left_on=['C.SUBJECT_ID'],right_on=['SUBJECT_ID'],how='left')
all_data1

,C.SUBJECT_ID,II.INTIME,C.HADM_ID,C.ICUSTAY_ID,D.ITEMID,C.CHARTTIME,C.VALUE,C.VALUENUM,C.VALUEUOM,II.OUTTIME,...,D.UNITNAME,hrs_bd,final_24hr,final_48hr,final_72hr,Death,VARIABLE,SUBJECT_ID,age,GENDER
0,109,2141-06-11 10:17:54,147469,253139,220045,2141-06-11 12:13:00,93.0,93.0,bpm,2141-06-15 15:11:37,...,bpm,98.976944,1,1,1,0,hr,109,20,F
1,109,2141-06-11 10:17:54,147469,253139,220045,2141-06-11 12:15:00,97.0,97.0,bpm,2141-06-15 15:11:37,...,bpm,98.943611,1,1,1,0,hr,109,20,F
2,109,2141-06-11 10:17:54,147469,253139,220045,2141-06-11 12:30:00,105.0,105.0,bpm,2141-06-15 15:11:37,...,bpm,98.693611,1,1,1,0,hr,109,20,F
3,109,2141-06-11 10:17:54,147469,253139,220045,2141-06-11 12:46:00,84.0,84.0,bpm,2141-06-15 15:11:37,...,bpm,98.426944,1,1,1,0,hr,109,20,F
4,109,2141-06-11 10:17:54,147469,253139,220045,2141-06-11 13:00:00,84.0,84.0,bpm,2141-06-15 15:11:37,...,bpm,98.193611,1,1,1,0,hr,109,20,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2207256,99992,2144-07-25 18:04:42,197084,242052,223791,2144-07-27 00:00:00,Mild,2.0,NaN,2144-07-27 17:27:55,...,NaN,17.465278,0,1,1,0,pain,99992,66,F
2207257,99992,2144-07-25 18:04:42,197084,242052,223791,2144-07-27 04:00:00,Mild,2.0,NaN,2144-07-27 17:27:55,...,NaN,13.465278,0,1,1,0,pain,99992,66,F
2207258,99992,2144-07-25 18:04:42,197084,242052,223791,2144-07-27 08:00:00,Mild,2.0,NaN,2144-07-27 17:27:55,...,NaN,9.465278,0,1,1,0,pain,99992,66,F
2207259,99992,2144-07-25 18:04:42,197084,242052,223791,2144-07-27 12:00:00,None,0.0,NaN,2144-07-27 17:27:55,...,NaN,5.465278,0,1,1,0,pain,99992,66,F


In [125]:
all_data1 = all_data1.drop(columns='SUBJECT_ID')
all_data1.to_csv('FirstICU_MinAge.csv')

In [126]:
all_data1 = pd.read_csv('FirstICU_MinAge.csv',index_col=0)

E:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [127]:
## 查看一下
print(len(all_data1[all_data1['C.SUBJECT_ID']==109]))  # 6206  条数据
print(len(all_data1[all_data1['C.SUBJECT_ID']==109]['D.ITEMID'].drop_duplicates()))  # 35 个检测变量
all_data1[all_data1['C.SUBJECT_ID']==109]['D.ITEMID'].drop_duplicates()   # 35 条数据

579
31


0      220045
108    220210
217    223761
242    220277
350    220587
351    220602
360    220615
368    220621
376    220644
377    220645
386    225624
394    225690
396    227442
405    227443
413    220228
420    220545
428    220546
435    225643
436    227457
443    227465
449    227466
455    227467
461    227464
462    220052
473    220050
485    220224
486    220235
487    220739
513    223900
539    223901
565    223791
Name: D.ITEMID, dtype: int64

In [128]:
all_data1['C.SUBJECT_ID'].drop_duplicates()

0            109
579          111
689          124
4574         188
4684         222
           ...  
2204802    99899
2206060    99923
2206153    99939
2206725    99973
2207044    99992
Name: C.SUBJECT_ID, Length: 3271, dtype: int64

In [129]:
all_data1[all_data1['GENDER']=='F']['C.SUBJECT_ID'].drop_duplicates()  # 1450

0            109
579          111
4684         222
4971         249
5210         305
           ...  
2186272    99715
2195227    99747
2198067    99847
2206725    99973
2207044    99992
Name: C.SUBJECT_ID, Length: 1450, dtype: int64

In [130]:
all_data1[all_data1['GENDER']=='M']['C.SUBJECT_ID'].drop_duplicates() # 1821

689          124
4574         188
5487         357
11780        665
12439        711
           ...  
2204378    99893
2204624    99897
2204802    99899
2206060    99923
2206153    99939
Name: C.SUBJECT_ID, Length: 1821, dtype: int64

In [131]:
all_data1.columns

Index(['C.SUBJECT_ID', 'II.INTIME', 'C.HADM_ID', 'C.ICUSTAY_ID', 'D.ITEMID',
       'C.CHARTTIME', 'C.VALUE', 'C.VALUENUM', 'C.VALUEUOM', 'II.OUTTIME',
       'II.LOS', 'D.LABEL', 'D.UNITNAME', 'hrs_bd', 'final_24hr', 'final_48hr',
       'final_72hr', 'Death', 'VARIABLE', 'age', 'GENDER'],
      dtype='object')

In [133]:
all_data1[all_data1['GENDER'].isnull()]

,C.SUBJECT_ID,II.INTIME,C.HADM_ID,C.ICUSTAY_ID,D.ITEMID,C.CHARTTIME,C.VALUE,C.VALUENUM,C.VALUEUOM,II.OUTTIME,...,D.LABEL,D.UNITNAME,hrs_bd,final_24hr,final_48hr,final_72hr,Death,VARIABLE,age,GENDER


In [134]:
all_data1 = all_data1[['C.SUBJECT_ID','C.ICUSTAY_ID','age','GENDER']]

In [135]:
all_data1.to_csv('age_gender_ICUid.csv',index=0)